# Cloudformation - autoscalingv2
`Status: COMPLETE`

Simplified template composed of:
- VPC `(10.0.0.0/16)`
 - IGW with route from `0.0.0.0/0`
- 2 Public Subnets
 - `10.0.1.0/24`
 - `10.0.1.0/24`
- AutoScalingGroup
 - Min:2, Max:2
- Application ELB
 - Healthchecks

Serving a page identifying instanceID over http/80

---

In [1]:
from troposphere import Template, Ref, GetAtt, GetAZs, Base64, Join, FindInMap, Select, Output, Parameter, Tags
from troposphere.autoscaling import AutoScalingGroup, LaunchConfiguration
from troposphere.autoscaling import Tag as asTag
from troposphere.autoscaling import Tags as asTags
import troposphere.ec2 as ec2
import troposphere.elasticloadbalancingv2 as elb

In [2]:
resources = set()
t = Template()
t.add_version('2010-09-09')
t.add_description('simple autoscaling, application elb')
# common tag to apply to most resources
site_tags = Tags(Site='example.com')
az1 = Select('0', GetAZs(Ref('AWS::Region')))
az2 = Select('1', GetAZs(Ref('AWS::Region')))

# Parameters
---

In [3]:
keyname = Parameter('keyname', 
                    Default='main', 
                    Description='Key to use for ssh access',
                    Type='String')
t.add_parameter(keyname);

# Maps

In [4]:
t.add_mapping('RegionMap', { 'us-east-1': {'ami': 'ami-b73b63a0'}
                           , 'us-east-2': {'ami': 'ami-58277d3d'}
                           , 'us-west-1': {'ami': 'ami-23e8a343'}
                           , 'us-west-2': {'ami': 'ami-5ec1673e'}
                           });

# Resources
---

## Network

### VPC

In [5]:
vpc = ec2.VPC('exampleVpc', CidrBlock='10.0.0.0/16', Tags=site_tags)
igw = ec2.InternetGateway('igw', Tags=site_tags)
attach_igw = ec2.VPCGatewayAttachment('attachIgw', InternetGatewayId=Ref(igw), VpcId=Ref(vpc))

resources.add(vpc)
resources.add(igw)
resources.add(attach_igw)

### Subnets

Two public subnets/AZs for use with application loadbalancer.

In [6]:
public1 = ec2.Subnet('public1',
                     AvailabilityZone=az1,
                     CidrBlock='10.0.1.0/24',
                     VpcId=Ref(vpc),
                     Tags=site_tags)
public2 = ec2.Subnet('public2',
                     AvailabilityZone=az2,
                     CidrBlock='10.0.2.0/24',
                     VpcId=Ref(vpc),
                     Tags=site_tags)
resources.add(public1)
resources.add(public2)

### Security Groups

In [7]:
allowed_ports = [22, 80]
rules = [ec2.SecurityGroupRule(IpProtocol='tcp', CidrIp='0.0.0.0/0', FromPort=p, ToPort=p) for p in allowed_ports]

web_dmz_sg = ec2.SecurityGroup('webDmzSg',
                               GroupDescription='allow ssh and http traffic',
                               SecurityGroupIngress=rules,
                               VpcId=Ref(vpc),
                               Tags=site_tags)
resources.add(web_dmz_sg)

### Routes

In [8]:
route_table = ec2.RouteTable('routeTable', VpcId=Ref(vpc), Tags=site_tags)
route_to_igw = ec2.Route('routeToIgw',
                         DestinationCidrBlock='0.0.0.0/0',
                         RouteTableId=Ref(route_table),
                         GatewayId=Ref(igw))
associate_route_to_public1 = ec2.SubnetRouteTableAssociation('associateToPublic1', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public1))
associate_route_to_public2 = ec2.SubnetRouteTableAssociation('associateToPublic2', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public2))
resources.add(route_table)
resources.add(route_to_igw)
resources.add(associate_route_to_public1)
resources.add(associate_route_to_public2)

## LoadBalancing

In [9]:
web_target_group = elb.TargetGroup('webTargetGroup',
                                   Name='webNodes',
                                   Port=80,
                                   Protocol='HTTP',
                                   HealthCheckPath='/',
                                   HealthCheckPort='80',
                                   HealthCheckProtocol='HTTP',
                                   HealthCheckIntervalSeconds=60,
                                   HealthCheckTimeoutSeconds=45,
                                   HealthyThresholdCount=4,
                                   UnhealthyThresholdCount=3,
                                   Matcher=elb.Matcher(HttpCode='200'),
                                   VpcId=Ref(vpc),
                                   Tags=site_tags)

loadbalancer = elb.LoadBalancer('webElb',
                                Name='webElb',
                                Scheme='internet-facing',
                                SecurityGroups=[Ref(web_dmz_sg)],
                                Subnets=[Ref(public1), Ref(public2)],
                                Tags=site_tags)

elb_listener = elb.Listener('listen80',
                            DefaultActions=[elb.Action(Type='forward', TargetGroupArn=Ref(web_target_group))],
                            LoadBalancerArn=Ref(loadbalancer),
                            Port=80,
                            Protocol='HTTP')



resources.add(elb_listener)
resources.add(loadbalancer)
resources.add(web_target_group)

## AutoScaling

In [10]:
script = """\
#!/bin/bash
sudo yum update -y
sudo yum install httpd -y
wget https://raw.githubusercontent.com/epequeno/aws-cf/master/make_page.py
sudo python make_page.py
sudo service httpd start
"""

def make_userdata(text):
    return Base64(Join('', text.splitlines(True)))

user_data = make_userdata(script)

launch_config = LaunchConfiguration('webLaunchConfig',
                                    AssociatePublicIpAddress=True,
                                    ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                                    InstanceType='t2.micro',
                                    KeyName=Ref(keyname),
                                    SecurityGroups=[GetAtt(web_dmz_sg, 'GroupId')],
                                    UserData=user_data)

asg_tag = asTag('Site', 'example.com', True)
asg = AutoScalingGroup('webAsg',
                       LaunchConfigurationName=Ref(launch_config),
                       AvailabilityZones=[az1, az2],
                       VPCZoneIdentifier=[Ref(public1), Ref(public2)],
                       TargetGroupARNs=[Ref(web_target_group)],
                       MinSize=2,
                       MaxSize=4,
                       Tags=[asg_tag])

resources.add(launch_config)
resources.add(asg)

# Outputs

In [11]:
elb_dns_name = Output('elbDnsName',
                      Description='DNS name for elb',
                      Value=GetAtt(loadbalancer, 'DNSName'))
t.add_output(elb_dns_name);

#  Build
---

In [12]:
for r in resources:
    t.add_resource(r)
    
with open('asgv2.json', 'w') as fd:
    fd.write(t.to_json())

## Validate

In [13]:
!aws cloudformation validate-template --template-body file://asgv2.json > /dev/null && echo "ok"

ok
